In [9]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from scripts.bloom_gen import BloomQuestionGenerator
from scripts.chunk import TextChunker

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test Bloom Question Generation

In [10]:
with open('../data/AI_Articles/Artificial intelligence.txt', 'r') as file:
    text = file.read()

In [27]:
chunker = TextChunker()
chunks = chunker.recursive_chunk(text, chunk_size=256, by_tokens=True)
print(len(chunks))
print(chunks[0])

106
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once something becomes useful enough and common e

In [30]:
bloom_generator = BloomQuestionGenerator()
question = bloom_generator.generate_question(chunks[0], question_type="MCQ", level=5, prompt_type="basic")
print(question)

Which of the following statements best evaluates the impact of artificial intelligence in everyday applications?

A. AI is only used in high-profile applications like autonomous vehicles and virtual assistants.
B. AI applications are always recognized and labeled as AI by the general public.
C. AI has become so integrated and useful in everyday applications that it often goes unnoticed and is not labeled as AI.
D. AI cannot be used in general applications as it is a complex field of research limited to computer systems.


In [31]:
question = bloom_generator.generate_question(chunks[0], question_type="SAQ", level=2, prompt_type="desc")
print(question)

Can you explain why some applications of artificial intelligence are not always recognized or labeled as such, despite their widespread use and utility?


In [33]:
question = bloom_generator.generate_question(chunks[0], question_type="SAQ", level=2, prompt_type="desc_examples")
print(question)

How would you describe the purpose and applications of Artificial Intelligence in everyday technology, such as web search engines, recommendation systems, and virtual assistants?


### Evalutate Generated Questions with BloomBERT